# Общая информация
Целью является создание модели прогнозирования энергопотребления просьюмеров для снижения затрат на небаланс энергии.

Проект направлен на решение проблемы энергетического дисбаланса - ситуации, когда ожидаемое потребление энергии не совпадает с ее фактическим использованием или производством. Значительная часть энергетического дисбаланса приходится на потребителей, которые как потребляют, так и производят энергию. Несмотря на то, что они составляют лишь небольшую часть всех потребителей, их непредсказуемое потребление энергии создает логистические и финансовые проблемы для энергетических компаний.
 
*Enefit* – одна из крупнейших энергетических компаний Балтийского региона. Как эксперты в области энергетики, мы помогаем клиентам индивидуально и гибко планировать свой экологический путь, а также реализовывать его, используя экологически чистые энергетические решения.

В настоящее время Enefit пытается решить проблему дисбаланса, разрабатывая внутренние модели прогнозирования и опираясь на прогнозы третьих сторон. Однако эти методы оказались недостаточными из-за низкой точности прогнозирования энергетического поведения просьюмеров. Недостатки этих нынешних методов заключаются в их неспособности точно учесть широкий спектр переменных, влияющих на поведение потребителей, что приводит к высоким издержкам дисбаланса.

### Оценивание
Основной метрикой заказчика является MAE (средняя абсолютная ошибка):

$MAE = \dfrac{1}{n} \sum_{i=1}^n |y_i - x_i|$

Где:

- $n$ — общее количество объектов
- $y_i$ — прогнозируемое значение таргета
- $x_i$ — наблюдаемое значение таргета

### Глоссарий
**Просьюмер** — 
1. человек, который покупает электронные товары, являющиеся стандартом между товарами, предназначенными для потребителей, и профессионалами
2. потребитель, который участвует в разработке или настройке продуктов для своих собственных нужд

В текущем проекте считаем просьюмером любого потребителя электроэнергии.

**EICs** — European Identifier Code.

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

# запишем путь директории
basedir = os.path.abspath(os.getcwd())

In [17]:
# загрузим данные
try:
    clients = pd.read_csv(os.path.join(basedir, 'data/client.csv'))
    train = pd.read_csv(os.path.join(basedir, 'data/train.csv'))
    gas_prices = pd.read_csv(os.path.join(basedir, 'data/gas_prices.csv'))
    electricity_prices = pd.read_csv(os.path.join(basedir, 'data/electricity_prices.csv'))
    forecast_weather = pd.read_csv(os.path.join(basedir, 'data/forecast_weather.csv'))
    historical_weather = pd.read_csv(os.path.join(basedir, 'data/historical_weather.csv'))
except:
    raise Exception('Check the files')

In [8]:
def get_dataset_info(df: pd.DataFrame) -> None:
    """
    Функция выводит общую инфу
    о датасете одним полем
    """
    display(df.head())
    print('\n')
    print(df.info())
    print('\n')
    print(df.isna().sum())
    print('\n')
    print(f'Дубликаты: {df.duplicated().sum()}')
    print('it`s over')

In [9]:
get_dataset_info(clients)

,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
0,1,0,108,952.89,0,2021-09-01,2
1,2,0,17,166.40,0,2021-09-01,2
2,3,0,688,7207.88,0,2021-09-01,2
3,0,0,5,400.00,1,2021-09-01,2
4,1,0,43,1411.00,1,2021-09-01,2




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41919 entries, 0 to 41918
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_type        41919 non-null  int64  
 1   county              41919 non-null  int64  
 2   eic_count           41919 non-null  int64  
 3   installed_capacity  41919 non-null  float64
 4   is_business         41919 non-null  int64  
 5   date                41919 non-null  object 
 6   data_block_id       41919 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 2.2+ MB
None


product_type          0
county                0
eic_count             0
installed_capacity    0
is_business           0
date                  0
data_block_id         0
dtype: int64


Дубликаты: 0
it`s over


Видим, что в этом датасете у нас 41919 объектов с обезличенной информацией о клиентах. Пропусков и полных дубликатов нет. Присутствуют следующие поля:
- **product_type** — тип продукта
- **county** — ID округа
- **eic_count** — количество точек потребления
- **installed_capacity** — установленная мощность фотоэлектрической солнечной панели в киловаттах
- **is_business** — является ли просьюмер бизнесом
- **date** — дата (вероятно, создания записи)
- **data_block_id** — дата (вероятно, блокировки пользователя или расторжения договора услуг)

In [13]:
get_dataset_info(train)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   county              int64  
 1   is_business         int64  
 2   product_type        int64  
 3   target              float64
 4   is_consumption      int64  
 5   datetime            object 
 6   data_block_id       int64  
 7   row_id              int64  
 8   prediction_unit_id  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 138.6+ MB
None


county                  0
is_business             0
product_type            0
target                528
is_consumption          0
datetime                0
data_block_id           0
row_id                  0
prediction_unit_id      0
dtype: int64


Дубликаты: 0
it`s over


В трейне у нас чуть более 2 миллионов объектов, у которых 528 пропусков в таргете. Учитывая, что их очень мало, эти строки просто удалим позже.
Поля:
- **county** — идентификационный код округа.
- **is_business** — логическое значение, определяющее, является ли просьюмер бизнесом.
- **product_type** — идентификационный код со следующим сопоставлением кодов типам контрактов: 
    - 0: «Комбинированный»
    - 1: «Фиксированный»
    - 2: «Общий сервис»
    - 3: «Спотовый»
- **target** — объем потребления или производства для соответствующего сегмента за час. Сегменты определяются округом, is_business и Product_type.
- **is_consumption** — логическое значение, указывающее, является ли целью этой строки потребление или производство.
- **datetime** — эстонское время в формате EET (UTC+2) / EEST (UTC+3).
- **data_block_id** — все строки с одним и тем же data_block_id будут доступны в одно и то же время прогноза. Это зависит от того, какая информация доступна во время фактических прогнозов, каждое утро в 11 часов утра. Например, если прогноз погоды data_block_id для предсказаний, сделанных 31 октября, равен 100, то исторический data_block_id погоды на 31 октября будет равен 101, поскольку исторические данные о погоде фактически доступны только на следующий день.
- **row_id** — уникальный идентификатор строки.
- **prediction_unit_id** — уникальный идентификатор для комбинации округа, is_business и Product_type. Новые единицы прогнозирования могут появляться или исчезать в тестовом наборе.

In [14]:
get_dataset_info(gas_prices)

,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
0,2021-09-01,45.23,46.32,2021-08-31,1
1,2021-09-02,45.62,46.29,2021-09-01,2
2,2021-09-03,45.85,46.40,2021-09-02,3
3,2021-09-04,46.30,46.80,2021-09-03,4
4,2021-09-05,46.30,46.58,2021-09-04,5




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   forecast_date          637 non-null    object 
 1   lowest_price_per_mwh   637 non-null    float64
 2   highest_price_per_mwh  637 non-null    float64
 3   origin_date            637 non-null    object 
 4   data_block_id          637 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 25.0+ KB
None


forecast_date            0
lowest_price_per_mwh     0
highest_price_per_mwh    0
origin_date              0
data_block_id            0
dtype: int64


Дубликаты: 0
it`s over


Здесь у нас информация по прогнозу цен на топливо.
- **origin_date** — дата, когда стали доступны цены на сутки вперед.
- **forecast_date**  — дата, когда прогнозные цены должны быть актуальными.
- **(lowest/highest)_price_per_mwh** — самая низкая/самая высокая цена природного газа, которая будет продаваться на день вперед в этот торговый день, в евро за эквивалент мегаватт-часа.
- **data_block_id**

In [15]:
get_dataset_info(electricity_prices)

,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15286 entries, 0 to 15285
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   forecast_date  15286 non-null  object 
 1   euros_per_mwh  15286 non-null  float64
 2   origin_date    15286 non-null  object 
 3   data_block_id  15286 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 477.8+ KB
None


forecast_date    0
euros_per_mwh    0
origin_date      0
data_block_id    0
dtype: int64


Дубликаты: 0
it`s over


Данные по ценам на электричество (почти аналогично газу):

- **origin_date**
- **forecast_date**
- **euros_per_mwh** — цена на электроэнергию на завтрашний день торгуется в евро за мегаватт-час.
- **data_block_id**

In [16]:
get_dataset_info(forecast_weather)

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,57.6,21.7,2021-09-01 00:00:00+00:00,1,15.655786,11.553613,0.904816,0.019714,0.000000,0.905899,-0.411328,-9.106137,1,2021-09-01 01:00:00+00:00,0.0,0.0,0.0,0.0
1,57.6,22.2,2021-09-01 00:00:00+00:00,1,13.003931,10.689844,0.886322,0.004456,0.000000,0.886658,0.206347,-5.355405,1,2021-09-01 01:00:00+00:00,0.0,0.0,0.0,0.0
2,57.6,22.7,2021-09-01 00:00:00+00:00,1,14.206567,11.671777,0.729034,0.005615,0.000000,0.730499,1.451587,-7.417905,1,2021-09-01 01:00:00+00:00,0.0,0.0,0.0,0.0
3,57.6,23.2,2021-09-01 00:00:00+00:00,1,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,1,2021-09-01 01:00:00+00:00,0.0,0.0,0.0,0.0
4,57.6,23.7,2021-09-01 00:00:00+00:00,1,15.293848,12.458887,0.102875,0.088074,0.000015,0.176590,1.268481,-8.975766,1,2021-09-01 01:00:00+00:00,0.0,0.0,0.0,0.0




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3424512 entries, 0 to 3424511
Data columns (total 18 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   latitude                           float64
 1   longitude                          float64
 2   origin_datetime                    object 
 3   hours_ahead                        int64  
 4   temperature                        float64
 5   dewpoint                           float64
 6   cloudcover_high                    float64
 7   cloudcover_low                     float64
 8   cloudcover_mid                     float64
 9   cloudcover_total                   float64
 10  10_metre_u_wind_component          float64
 11  10_metre_v_wind_component          float64
 12  data_block_id                      int64  
 13  forecast_datetime                  object 
 14  direct_solar_radiation             float64
 15  surface_solar_radiation_downwards  float64
 16  snowfall        

Прогнозы погоды, которые были бы доступны на момент прогнозирования от Европейского центра среднесрочных прогнозов погоды. 3,5 миллионов записей.

- **latitude, longitude** — координаты прогноза погоды.
- **origin_datetime** — временная метка создания прогноза.
- **hours_ahead** — количество часов между созданием прогноза и прогнозом погоды. Каждый прогноз охватывает в общей сложности 48 часов.
- **temperature** — температура воздуха на высоте 2 метра над землей в градусах Цельсия.
- **dewpoint** — температура точки росы на высоте 2 метра над землей в градусах Цельсия.
- **cloudcover_(low/mid/high/total)** — процент неба, покрытого облаками в следующих диапазонах высот: 0–2 км, 2–6, 6+ и общий.
- **10_metre_(u/v)_wind_component** — составляющая скорости ветра (восточное/северное направление), измеренная на высоте 10 метров над поверхностью земли в метрах в секунду.
- **data_block_id**
- **forecast_datetime** — временная метка прогнозируемой погоды. Генерируется из origin_datetime плюс hours_ahead.
- **direct_solar_radiation** — прямая солнечная радиация, достигающая поверхности в плоскости, перпендикулярной направлению Солнца, накопленная за предыдущий час, в ватт-часах на квадратный метр.
- **surface_solar_radiation_downwards** — солнечное излучение, как прямое, так и рассеянное, достигающее горизонтальной плоскости на поверхности Земли, в ватт-часах на квадратный метр.
- **snowfall** — снегопад за предыдущий час в метрах водного эквивалента.
- **total_precipitation** — накопленная жидкость, состоящая из дождя и снега, выпавшая на поверхность Земли за предыдущий час, в метрах.

In [18]:
get_dataset_info(historical_weather)

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
0,2021-09-01 00:00:00,14.4,12.0,0.0,0.0,1015.8,4,4,0,0,6.694444,3,0.0,0.0,0.0,57.6,21.7,1
1,2021-09-01 00:00:00,14.0,12.0,0.0,0.0,1010.6,7,8,0,0,4.944444,353,0.0,0.0,0.0,57.6,22.2,1
2,2021-09-01 00:00:00,14.4,12.8,0.0,0.0,1014.9,6,7,0,0,5.833333,348,0.0,0.0,0.0,57.6,22.7,1
3,2021-09-01 00:00:00,15.4,13.0,0.0,0.0,1014.4,4,2,4,0,7.111111,349,0.0,0.0,0.0,57.6,23.2,1
4,2021-09-01 00:00:00,15.9,12.6,0.0,0.0,1013.8,12,7,0,20,8.388889,360,0.0,0.0,0.0,57.6,23.7,1




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710800 entries, 0 to 1710799
Data columns (total 18 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   datetime                object 
 1   temperature             float64
 2   dewpoint                float64
 3   rain                    float64
 4   snowfall                float64
 5   surface_pressure        float64
 6   cloudcover_total        int64  
 7   cloudcover_low          int64  
 8   cloudcover_mid          int64  
 9   cloudcover_high         int64  
 10  windspeed_10m           float64
 11  winddirection_10m       int64  
 12  shortwave_radiation     float64
 13  direct_solar_radiation  float64
 14  diffuse_radiation       float64
 15  latitude                float64
 16  longitude               float64
 17  data_block_id           int64  
dtypes: float64(11), int64(6), object(1)
memory usage: 234.9+ MB
None


datetime                  0
temperature               0
dewpoint         

Исторические данные погоды. 

- **datetime**
- **temperature**
- **dewpoint**
- **rain** — разница от условных обозначений прогноза. Дождь от крупномасштабных погодных систем за предыдущий час в миллиметрах.
- **snowfall** — разница от условного прогноза. Снегопад за предыдущий час в сантиметрах.
- **surface_pressure** — давление воздуха на поверхности в гектопаскалях.
- **cloudcover_(low/mid/high/total)** — разница от прогноза. Облачность на высоте 0–3 км, 3–8, 8+ и общая.
- **windspeed_10m** — разница от прогноза. Скорость ветра на высоте 10 метров над землей в метрах в секунду.
- **winddirection_10m** — разница от прогноза. Направление ветра на высоте 10 метров над землей в градусах.
- **shortwave_radiation** — разница от прогноза. Глобальное горизонтальное излучение в ватт-часах на квадратный метр.
- **direct_solar_radiation**
- **diffuse_radiation** — разница от прогноза. Диффузное солнечное излучение в ватт-часах на квадратный метр.
- **(latitude/longitude)** — координаты метеостанции.
- **data_block_id**